In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

Passos desse notebook

1. Merge de datasets

2. Transformar colunas

- Variáveis Categóricas (Partido, Tema, RegimeTramitacao, Ano)
    - One-Hot Encoding
- Variáveis Numéricas (Quantidade de votos do partido X)
    - Normalizar

In [3]:
ListaProposicoes = os.path.join('data', 'ListaProposicoes_2013_2024.csv')
ListaProposicoesVotadas_Plenario = os.path.join('data', 'ListaProposicoesVotadasPlenario_2013_2024.csv')
ObterProposicoes = os.path.join('data', 'ObterProposicoes_2013_2024.csv')
ObterProposicoes_2013 = os.path.join('data', 'obter_proposicoes' ,'ObterProposicoes_2013.csv')
VotacoesProposicoes = os.path.join('data', 'VotacoesProposicoes_2013_2024.csv')

In [4]:
df = pd.read_csv(ObterProposicoes)
df_pvp = pd.read_csv(ListaProposicoesVotadas_Plenario)
df_vp = pd.read_csv(VotacoesProposicoes)
df_lp = pd.read_csv(ListaProposicoes)

## Removendo duplicatas

Devido ao processamento paralelo das requisições, alguns dados foram salvos mais de uma vez com base no ID. Para garantir a integridade dos dados, as duplicatas serão eliminadas.


In [5]:
duplicated_ids = df[df['idProposicao'].duplicated()]['idProposicao']
#print(duplicated_ids)

if duplicated_ids.any():
    df = df.drop_duplicates(subset='idProposicao')

#print(df)

In [6]:
df.columns

Index(['nomeProposicao', 'idProposicao', 'idProposicaoPrincipal',
       'nomeProposicaoOrigem', 'tipoProposicao', 'tema', 'Ementa',
       'ExplicacaoEmenta', 'Autor', 'ideCadastro', 'ufAutor', 'partidoAutor',
       'DataApresentacao', 'RegimeTramitacao', 'UltimoDespacho', 'Apreciacao',
       'Indexacao', 'Situacao', 'LinkInteiroTeor', 'apensadas',
       'apensadas_proposicao'],
      dtype='object')

Colunas selecionadas para ficar:

In [7]:
df = df[["nomeProposicao", "idProposicao", "tema", "partidoAutor", "DataApresentacao", "RegimeTramitacao","Situacao"]]

In [8]:
df

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,RegimeTramitacao,Situacao
0,PL 7024/2013,605259,Administração Pública,\n,20/12/2013,"Prioridade (Art. 151, II, RICD)",MESA - Arquivada
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,20/12/2013,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,20/12/2013,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto
3,PL 7027/2013,605291,"Política, Partidos e Eleições",\n,20/12/2013,"Prioridade (Art. 151, II, RICD)",Tranformada no(a) Lei Ordinária 13150/2015
4,PL 7029/2013,605307,Educação,PT,23/12/2013,"Ordinário (Art. 151, III, RICD)",CFT - Aguardando Designação de Relator(a)
...,...,...,...,...,...,...,...
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,10/01/2024,"Prioridade (Art. 151, II, RICD)",MESA - Aguardando Apreciação pelo Senado Federal
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,02/02/2024,"Prioridade (Art. 151, II, RICD)",CDE - Aguardando Parecer
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,05/02/2024,"Prioridade (Art. 151, II, RICD)",CPASF - Tramitando em Conjunto
73841,PL 2/2024,2416804,Economia; Finanças Públicas e Orçamento; Indús...,\n,03/01/2024,"Urgência (Art. 64, CF)",Tranformada no(a) Lei Ordinária 14871/2024


In [26]:
df['RegimeTramitacao'].unique()

array(['Prioridade (Art. 151, II, RICD)',
       'Ordinário (Art. 151, III, RICD)', 'Urgência (Art. 155, RICD)',
       '.', 'Urgência (Art. 154, RICD)',
       'Especial (Arts. 142 e 143, RCCN)', 'Especial',
       'Urgência (Art. 64, CF)', 'Especial (Art. 213, § 6º, RICD)',
       'Urgência', 'Especial (Art. 207, RICD)', '\n  '], dtype=object)

In [21]:
df_teste = df

In [22]:
# Extrair a letra inicial da categoria
df_teste['categoria'] = df['RegimeTramitacao'].str.extract(r'([EPUO])')

# Criar colunas one-hot encoded com base na categoria extraída
df_one_hot = pd.get_dummies(df_teste['categoria'])

# Juntar as colunas one-hot encoded ao DataFrame original
df_teste = df_teste.join(df_one_hot)

In [23]:
df_teste

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,RegimeTramitacao,Situacao,categoria,E,O,P,U
0,PL 7024/2013,605259,Administração Pública,\n,20/12/2013,"Prioridade (Art. 151, II, RICD)",MESA - Arquivada,P,False,False,True,False
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,20/12/2013,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto,P,False,False,True,False
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,20/12/2013,"Prioridade (Art. 151, II, RICD)",CCJC - Tramitando em Conjunto,P,False,False,True,False
3,PL 7027/2013,605291,"Política, Partidos e Eleições",\n,20/12/2013,"Prioridade (Art. 151, II, RICD)",Tranformada no(a) Lei Ordinária 13150/2015,P,False,False,True,False
4,PL 7029/2013,605307,Educação,PT,23/12/2013,"Ordinário (Art. 151, III, RICD)",CFT - Aguardando Designação de Relator(a),O,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,10/01/2024,"Prioridade (Art. 151, II, RICD)",MESA - Aguardando Apreciação pelo Senado Federal,P,False,False,True,False
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,02/02/2024,"Prioridade (Art. 151, II, RICD)",CDE - Aguardando Parecer,P,False,False,True,False
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,05/02/2024,"Prioridade (Art. 151, II, RICD)",CPASF - Tramitando em Conjunto,P,False,False,True,False
73841,PL 2/2024,2416804,Economia; Finanças Públicas e Orçamento; Indús...,\n,03/01/2024,"Urgência (Art. 64, CF)",Tranformada no(a) Lei Ordinária 14871/2024,U,False,False,False,True


In [24]:
df_teste.rename(columns={
    'E': 'Especial',
    'O': 'Ordinário',
    'P': 'Prioridade',
    'U': 'Urgência'
}, inplace=True)

df_teste.drop(columns=['RegimeTramitacao', 'categoria'], inplace=True)

df_teste.replace({True: 1, False: 0}, inplace=True)

C:\Users\kamil\AppData\Local\Temp\ipykernel_16540\3101857384.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_teste.replace({True: 1, False: 0}, inplace=True)


In [25]:
df_teste

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,Situacao,Especial,Ordinário,Prioridade,Urgência
0,PL 7024/2013,605259,Administração Pública,\n,20/12/2013,MESA - Arquivada,0,0,1,0
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0
3,PL 7027/2013,605291,"Política, Partidos e Eleições",\n,20/12/2013,Tranformada no(a) Lei Ordinária 13150/2015,0,0,1,0
4,PL 7029/2013,605307,Educação,PT,23/12/2013,CFT - Aguardando Designação de Relator(a),0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,10/01/2024,MESA - Aguardando Apreciação pelo Senado Federal,0,0,1,0
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,02/02/2024,CDE - Aguardando Parecer,0,0,1,0
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,05/02/2024,CPASF - Tramitando em Conjunto,0,0,1,0
73841,PL 2/2024,2416804,Economia; Finanças Públicas e Orçamento; Indús...,\n,03/01/2024,Tranformada no(a) Lei Ordinária 14871/2024,0,0,0,1


In [31]:
df_teste_2 = df_teste

df_teste_2['categoria'] = df_teste['Situacao'].str.extract(r'([CMT])')

df_teste_2


,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,Situacao,Especial,Ordinário,Prioridade,Urgência,categoria
0,PL 7024/2013,605259,Administração Pública,\n,20/12/2013,MESA - Arquivada,0,0,1,0,M
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0,C
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0,C
3,PL 7027/2013,605291,"Política, Partidos e Eleições",\n,20/12/2013,Tranformada no(a) Lei Ordinária 13150/2015,0,0,1,0,T
4,PL 7029/2013,605307,Educação,PT,23/12/2013,CFT - Aguardando Designação de Relator(a),0,1,0,0,C
...,...,...,...,...,...,...,...,...,...,...,...
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,10/01/2024,MESA - Aguardando Apreciação pelo Senado Federal,0,0,1,0,M
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,02/02/2024,CDE - Aguardando Parecer,0,0,1,0,C
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,05/02/2024,CPASF - Tramitando em Conjunto,0,0,1,0,C
73841,PL 2/2024,2416804,Economia; Finanças Públicas e Orçamento; Indús...,\n,03/01/2024,Tranformada no(a) Lei Ordinária 14871/2024,0,0,0,1,T


In [37]:

palavras_chave = ['transformada', 'Tramitando', 'aguardando', 'arquivada']
regex = '|'.join(palavras_chave)

df_filtrado = df_teste_2[df_teste_2['Situacao'].str.contains(regex, case=False, na=False)]
df_filtrado

,nomeProposicao,idProposicao,tema,partidoAutor,DataApresentacao,Situacao,Especial,Ordinário,Prioridade,Urgência,categoria
0,PL 7024/2013,605259,Administração Pública,\n,20/12/2013,MESA - Arquivada,0,0,1,0,M
1,PL 7025/2013,605279,Direito Penal e Processual Penal; Direitos Hum...,PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0,C
2,PL 7026/2013,605290,"Política, Partidos e Eleições",PT,20/12/2013,CCJC - Tramitando em Conjunto,0,0,1,0,C
4,PL 7029/2013,605307,Educação,PT,23/12/2013,CFT - Aguardando Designação de Relator(a),0,1,0,0,C
5,PL 7022/2013,604998,Direito e Defesa do Consumidor,PSD,19/12/2013,PLEN - Tramitando em Conjunto,0,0,1,0,T
...,...,...,...,...,...,...,...,...,...,...,...
73837,PL 20/2024,2416869,Direito Penal e Processual Penal,UNIÃO,05/02/2024,CPASF - Tramitando em Conjunto,0,1,0,0,C
73838,PL 3/2024,2416826,Direito Civil e Processual Civil; Economia,\n,10/01/2024,MESA - Aguardando Apreciação pelo Senado Federal,0,0,1,0,M
73839,PL 15/2024,2416861,Finanças Públicas e Orçamento,\n,02/02/2024,CDE - Aguardando Parecer,0,0,1,0,C
73840,PL 17/2024,2416863,Direito Penal e Processual Penal,PATRIOTA,05/02/2024,CPASF - Tramitando em Conjunto,0,0,1,0,C


In [34]:
primeiras_letras = df_teste_2['Situacao'].str.split().str[0].unique()

print(primeiras_letras)

['MESA' 'CCJC' 'Tranformada' 'CFT' 'PLEN' 'CSAUDE' 'CPASF' 'Diversas' 'CE'
 'CASP' 'CTRAB' 'CAPADR' 'CDC' 'CDHMIR' 'CME' 'CCP' 'CMULHER' 'CPOVOS'
 'CMADS' 'CDE' 'CVT' 'ARQUIVO' 'SERAU(SGM)' 'CREDN' 'SGM' 'CCOM' 'MESA-CN'
 'CCTI' 'CICS' 'CDU' 'GTCL' 'CCULT' 'CPD' 'Originou' 'NAJUR(SGM)' 'CTUR'
 'CINDRE' 'CESPO' 'CSPCCO' '1SECM' 'SECAP(SGM)' 'SGMGAB' 'CIDOSO' 'SIDEX'
 'CD']


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Listas das categorias
direitos_justica = [
    'Direito Penal e Processual Penal', 'Direito e Justiça', 
    'Direito Civil e Processual Civil', 'Direito Constitucional', 
    'Direito e Defesa do Consumidor', 'Direitos Humanos e Minorias'
]

desenvolvimento_social_urbano = [
    'Cidades e Desenvolvimento Urbano', 'Saúde', 'Educação', 
    'Trabalho e Emprego', 'Arte, Cultura e Religião', 
    'Esporte e Lazer', 'Previdência e Assistência Social', 
    'Homenagens e Datas Comemorativas'
]

economia_desenvolvimento_sustentavel = [
    'Meio Ambiente e Desenvolvimento Sustentável', 'Economia', 
    'Indústria, Comércio e Serviços', 'Finanças Públicas e Orçamento', 
    'Agricultura, Pecuária, Pesca e Extrativismo', 'Estrutura Fundiária', 
    'Turismo', 'Relações Internacionais e Comércio Exterior', 
    'Energia, Recursos Hídricos e Minerais'
]

administracao_politicas_publicas = [
    'Administração Pública', 'Processo Legislativo e Atuação Parlamentar', 
    'Política, Partidos e Eleições', 'Defesa e Segurança', 
    'Comunicações', 'Viação, Transporte e Mobilidade', 
    'Ciência, Tecnologia e Inovação'
]

infraestrutura_gestao_recursos = [
    'Viação, Transporte e Mobilidade', 'Energia, Recursos Hídricos e Minerais', 
    'Estrutura Fundiária', 'Comunicações'
]

# Função para categorizar temas
def categorizar_tema(tema):
    if tema in direitos_justica:
        return 'Direitos e Justiça'
    elif tema in desenvolvimento_social_urbano:
        return 'Desenvolvimento Social e Urbano'
    elif tema in economia_desenvolvimento_sustentavel:
        return 'Economia e Desenvolvimento Sustentável'
    elif tema in administracao_politicas_publicas:
        return 'Administração e Políticas Públicas'
    elif tema in infraestrutura_gestao_recursos:
        return 'Infraestrutura e Gestão de Recursos'
    else:
        return 'Outros'

# Supondo que você tenha um dataframe df com a coluna 'tema'
df['categoria'] = df['tema'].apply(categorizar_tema)

# Realizando o One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(df[['categoria']])

# Convertendo para dataframe e adicionando ao original
categorias_encoded = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.categories_[0])
df = pd.concat([df, categorias_encoded], axis=1)

# Exibe o dataframe resultante
print(df)